# Sports news generator

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from html2text import html2text
import re
from nltk import word_tokenize
import datetime
import locale
import time
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = "https://peru21.pe/deportes/futbol-peruano/"

# r = requests.get(url)
# soup = BeautifulSoup(r.text, 'lxml')

driver.get(url)

In [42]:
elem_hrefs = driver.find_elements(by=By.XPATH, value='//a[contains(text(), "Fútbol peruano")]/parent::div/h2/a')
links = [ele.get_attribute("href") for ele in elem_hrefs]
links

['https://peru21.pe/deportes/futbol-peruano/premio-al-esfuerzo-piero-quispe-fue-elegido-como-el-mejor-jugador-de-la-liga-1-2023-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/a-festejar-crema-hoy-es-la-premiacion-de-universitario-como-campeon-nacional-2023-peru-universitario-campeon-futbol-peruano-liga-1-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/universitario-edison-flores-tienen-que-venir-mas-extranjeros-como-hernan-barcos-a-la-liga-1-titulo-de-la-u-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/premiacion-liga-1-quien-fue-el-mejor-jugador-del-futbol-peruano-este-ano-piero-quispe-universitario-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/universitario-jean-ferrari-si-queremos-a-yoshimar-yotun-video-refuerzos-2024-sporting-cristal-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/universitario-altas-y-bajas-para-el-proximo-ano-centenario-u-plantel-2024-refuerzos-renovaciones-noticia/',
 'https://peru21.pe/deportes/futbol-peruano/alianza-lima-

In [43]:
driver.get("https://peru21.pe/deportes/futbol-peruano/a-festejar-crema-hoy-es-la-premiacion-de-universitario-como-campeon-nacional-2023-peru-universitario-campeon-futbol-peruano-liga-1-noticia/")

In [44]:
foot_new = driver.find_elements(by=By.XPATH, value='//p')

In [45]:
full_text_new = ''
long_new = len(foot_new)
for idx, ele in enumerate(foot_new):
    if idx not in [0, long_new-1]:
        text_new = re.sub(r'[^a-zA-Z0-9Ññáéíóú,;.\s]', '', ele.text).strip()
        if text_new != '':
            full_text_new = full_text_new + " " + text_new

full_text_new = full_text_new.strip()

In [49]:
ele_title = driver.find_element(by=By.XPATH, value='//h1')
football_title = ele_title.text

In [50]:
football_title

'A festejar crema: así fue la premiación de Universitario como campeón nacional 2023'

In [51]:
ele_subtitle = driver.find_element(by=By.XPATH, value="//h2[contains(@class, 'sht__summary')]")
football_subtitle = ele_subtitle.text
football_subtitle

'En el Monumental de Ate, los merengues serán premiados luego de ganarle la final a Alianza Lima y coronándose como campeón del Perú esta temporada.'

## Web scraping News

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from html2text import html2text
import re
from nltk import word_tokenize
import datetime
import locale
import time
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

date_n = datetime.datetime.strptime("2023-11-03", "%Y-%m-%d")
date_u = datetime.datetime.strptime("2023-01-01", "%Y-%m-%d")

total_news = 0
links = list()

while True:

    try:
        day = date_n.strftime("%d")
        month = date_n.strftime("%m")
        year = date_n.strftime("%Y")

        url = "https://peru21.pe/archivo/todas/{}-{}-{}/".format(year, month, day)

        print(date_n.strftime("Scraping the news from %d/%m/%Y"))
        
        driver.get(url)

        for i in range(4):
            # scroll to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        elem_hrefs = driver.find_elements(by=By.XPATH, value='//a[contains(text(), "Fútbol peruano")]/parent::div/h2/a')
        if elem_hrefs:
            print("Found {} peruvian football news".format(len(elem_hrefs)))
            total_news = total_news + len(elem_hrefs)
        else:
            print("Not found peruvian football")

        for ele in elem_hrefs:
            links.append(ele.get_attribute("href"))

        date_n = date_n - datetime.timedelta(days=1)

        print("Correct scraping! Total news: {}".format(total_news))
        time.sleep(2)

        if date_n == date_u:
            break
        else:
            continue

    except Exception as e:
        print(e)
        print("Incorrect scraping!")

In [77]:
def check_text(text_new):
    text_new = re.sub(r'[^a-zA-Z0-9Ññáéíóú,;.\s]', '', text_new).strip()
    return text_new

In [78]:
df_football_news = pd.DataFrame()

elem_hrefs = driver.find_elements(by=By.XPATH, value='//a[contains(text(), "Fútbol peruano")]/parent::div/h2/a')
links = [ele.get_attribute("href") for ele in elem_hrefs]

for link in links:
    
    driver.get(link)

    time.sleep(5)

    foot_new = driver.find_elements(by=By.XPATH, value='//p')
    ele_title = driver.find_element(by=By.XPATH, value='//h1')
    ele_subtitle = driver.find_element(by=By.XPATH, value="//h2[contains(@class, 'sht__summary')]")

    full_text_new = ''
    long_new = len(foot_new)
    for idx, ele in enumerate(foot_new):
        if idx not in [0, long_new-1]:
            text_new = check_text(ele.text)
            if text_new != '':
                full_text_new = full_text_new + " " + text_new
    full_text_new = full_text_new.strip()

    football_title = ele_title.text
    football_title = check_text(football_title)

    
    football_subtitle = ele_subtitle.text
    football_subtitle = check_text(football_subtitle)

    df_football_news_c = pd.DataFrame({"Title": [football_title], "Subtitle": [football_subtitle], "Text": [full_text_new]})
    df_football_news = pd.concat([df_football_news, df_football_news_c], axis=0)

    print("Correct scraping: {}".format(link))

driver.close()


df_football_news.reset_index(drop=True, inplace=True)

Correct scraping: https://peru21.pe/deportes/futbol-peruano/premio-al-esfuerzo-piero-quispe-fue-elegido-como-el-mejor-jugador-de-la-liga-1-2023-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/a-festejar-crema-hoy-es-la-premiacion-de-universitario-como-campeon-nacional-2023-peru-universitario-campeon-futbol-peruano-liga-1-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-edison-flores-tienen-que-venir-mas-extranjeros-como-hernan-barcos-a-la-liga-1-titulo-de-la-u-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/premiacion-liga-1-quien-fue-el-mejor-jugador-del-futbol-peruano-este-ano-piero-quispe-universitario-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-jean-ferrari-si-queremos-a-yoshimar-yotun-video-refuerzos-2024-sporting-cristal-noticia/
Correct scraping: https://peru21.pe/deportes/futbol-peruano/universitario-altas-y-bajas-para-el-proximo-ano-centenario-u-plantel-2024-refue

In [ ]:
df_football_news['Text'] = df_football_news['Text'].apply(lambda x: x.replace("\n", ""))

In [98]:
df_football_news.head(5)

,Title,Subtitle,Text
0,Premio al esfuerzo Piero Quispe fue elegido co...,El mediocampista de Universitario fue pieza fu...,"Luego de 10 años de sequía, Universitario form..."
1,A festejar crema así fue la premiación de Univ...,"En el Monumental de Ate, los merengues serán p...",El equipo crema se dispone a dar la vuelta olí...
2,Se disculpó con Barcos Flores Tienen que venir...,Después de imitarlo en la celebración por el t...,En medio de las celebraciones por su nuevo tít...
3,Liga 1 Quién fue el mejor jugador del fútbol p...,Conoce qué futbolistas tuvieron los mejores nú...,"Sin embargo, saliendo de los gustos de cada un..."
4,Yotún jugará en Universitario Jean Ferrari Sí ...,"En el 2024, el jugador emblema de Sporting Cri...","Universitario ganó la Liga 1 este año, pero qu..."


In [96]:
df_football_news.to_csv("peruvian_football_new.csv", index=False, sep="|")

In [117]:
full_text_new = ''
for i in range(len(df_football_news)):
    full_text_new = full_text_new + df_football_news.loc[i, 'Title'] + ":" + "\n"
    full_text_new = full_text_new + df_football_news.loc[i, 'Text'] + "\n\n"

In [118]:
with open("peruvian_football_news.txt", "w") as f:
    f.write(full_text_new)

## GRU (Gated Recurrent Unit)

In [119]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [221]:
full_text_new = open("peruvian_football_news.txt", 'rb').read().decode(encoding='utf-8')

In [193]:
def replace_tild(text):
    accent_m = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'Á': 'A',
        'É': 'E',
        'Í': 'I',
        'Ó': 'O',
        'Ú': 'U'
    }
    for k,v in accent_m.items():
        text = text.replace(k,v) 
    return text

In [194]:
full_text_new = replace_tild(full_text_new)

In [196]:
print(full_text_new[:2000])

Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:
Luego de 10 años de sequia, Universitario formo un gran plantel para gritar campeon este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 años fue clave para el elenco dirigido por Jorge Fossati, que se llevo el titulo ante su clasico rival y este domingo recien pudo recibir el trofeo. El volante crema aporto a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimio un gran juego en colectivo a su equipo, apoyandolo en el mediocampo con jugadores como Rodrigo Ureña o Jose Perez Guedes, que eran un respaldo para Quispe. Pierito tambien destaco en la campaña de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buenas actuaciones le valieron la convocatoria a la seleccion nacional dirigida por Juan Reynoso, aunque no ha debutado oficialmente todavia. E

In [197]:
# The unique characters in the file
vocab = sorted(set(full_text_new))
print(f'{len(vocab)} unique characters')

69 unique characters


In [198]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [199]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [200]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [201]:
all_ids = ids_from_chars(tf.strings.unicode_split(full_text_new, 'UTF-8'))
all_ids

<tf.Tensor: shape=(83442,), dtype=int64, numpy=array([32, 60, 47, ...,  4,  1,  1], dtype=int64)>

In [202]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [203]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

P
r
e
m
i
o
 
a
l
 


In [204]:
seq_length = 1000
examples_per_epoch = len(full_text_new)//(seq_length+1) 

In [205]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor([b'P' b'r' b'e' ... b'n' b' ' b'd'], shape=(1667,), dtype=string)


In [206]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:\nLuego de 10 a\xc3\xb1os de sequia, Universitario formo un gran plantel para gritar campeon este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 a\xc3\xb1os fue clave para el elenco dirigido por Jorge Fossati, que se llevo el titulo ante su clasico rival y este domingo recien pudo recibir el trofeo. El volante crema aporto a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimio un gran juego en colectivo a su equipo, apoyandolo en el mediocampo con jugadores como Rodrigo Ure\xc3\xb1a o Jose Perez Guedes, que eran un respaldo para Quispe. Pierito tambien destaco en la campa\xc3\xb1a de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buenas actuaciones le valieron la convocatoria a la seleccion nacional dirigida por Juan Reynoso, aunque no ha d

In [207]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [208]:
dataset = sequences.map(split_input_target)

In [209]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:\nLuego de 10 a\xc3\xb1os de sequia, Universitario formo un gran plantel para gritar campeon este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 a\xc3\xb1os fue clave para el elenco dirigido por Jorge Fossati, que se llevo el titulo ante su clasico rival y este domingo recien pudo recibir el trofeo. El volante crema aporto a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimio un gran juego en colectivo a su equipo, apoyandolo en el mediocampo con jugadores como Rodrigo Ure\xc3\xb1a o Jose Perez Guedes, que eran un respaldo para Quispe. Pierito tambien destaco en la campa\xc3\xb1a de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buenas actuaciones le valieron la convocatoria a la seleccion nacional dirigida por Juan Reynoso, aunque

In [210]:
# Batch size
BATCH_SIZE = 16

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 1666), dtype=tf.int64, name=None), TensorSpec(shape=(64, 1666), dtype=tf.int64, name=None))>

In [211]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [212]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [213]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [214]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [215]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

### Train the model

In [182]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [184]:
model.compile(optimizer='adam', loss=loss)

In [186]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=20, callbacks=[checkpoint_callback])

In [189]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Universitario ganó la final:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)